<FONT size="5.9pt"> **NB2 - PROSPECTIVE CONSEQUENTIAL LCA OF BARK BIOREFINING - PART 2**
</FONT> 

This Part 2 focuses on calculating the time-dependent (or dynamic) climate impacts of the bark biorefinery and combustion scenarios using the dynamic characterisation factors (DCFs) over a 100-year time horizon developped by [Levasseur et al. (2010)](https://pubs.acs.org/doi/10.1021/es9030003) and based on the ALIGNED guidelines ([Hamelin et al., 2024](https://zenodo.org/records/10843259)).

In [ ]:
import os
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# define the initial working directory
wdir = Path().resolve() # get the directory of the notebook
pdir = os.path.join(wdir,'projects')
print(f"Working Directory: {wdir}")

# set the directory to store the results
result_dir = os.path.join(wdir,'results')   
pathlib.Path(result_dir).mkdir(parents=True, exist_ok=True)
print('Results directory:', result_dir)

# 1. Dynamic characterisation factors (from Levasseur et al. 2010)

In [ ]:
from scipy.interpolate import UnivariateSpline

# dynamic characterisation factors for 100 years time horizon developed by Levasseur et al. (2010)
DCF100 = pd.read_excel(str(str(wdir) + '/dynamic_characterisation_factors_Levasseur100.xlsx'))

x = DCF100.index
yCO2 = DCF100['CO2_CF']
yCH4 = DCF100['CH4_CF']
yN2O = DCF100['N2O_CF']
fCO2 = UnivariateSpline(x, yCO2, s=1)
fCH4 = UnivariateSpline(x, yCH4, s=1)
fN2O = UnivariateSpline(x, yN2O, s=1)

plt.scatter(x, yCO2, marker = '.',label = 'DCF CO2',color = 'black',zorder=1)
plt.scatter(x, yCH4, marker = '.',label = 'DCF CH4',color = 'blue',zorder=1)
plt.scatter(x, yN2O, marker = '.',label = 'DCF N2O',color = 'green',zorder=1)
#plt.plot(x, fCO2(x), '--',lw = 1, label = 'Spline Fit',color = 'red',zorder=2)

plt.xlabel('Time (year)', fontsize=13)
plt.ylabel('Dynamic characterisation factor', fontsize=13)


plt.legend(frameon=False)
plt.savefig(str(result_dir+'/SM1_DCFs.pdf'), dpi=300)
plt.show()

# 2. Time-dependent climate impacts from indirect CO2, CH4, and N2O emissions 

## 2.1. Import the indirect emissions inventory (CO2, CH4, and N2O)

The time-dependent climate impacts are calculated using the DCFs developed by Levasseur et al. (2010) and shown above. The inventory data for the three scenarios were obtained from the Activity Browser in order to have the list of indirect CO2, CH4, and N2O emissions (see SM3, Excel files). The indirect emissions were aggregated per main product/process. 

In [ ]:
sheet_names_bg = ['ei39_CO2','ei39_CH4','ei39_N2O', # ecoinvent consequential 3.9
                  'R2Pk_CO2','R2Pk_CH4','R2Pk_N2O', # # ecoinvent consequential 3.9 REMIND SSP2 PkBudg500 2045
                  'R2B_CO2','R2B_CH4','R2B_N2O', # # ecoinvent consequential 3.9 REMIND SSP2 Base 2045
                  'R1Pk_CO2','R1Pk_CH4','R1Pk_N2O', # # ecoinvent consequential 3.9 REMIND SSP1 PkBudg500 2045
                  'I2B_CO2','I2B_CH4','I2B_N2O', # # ecoinvent consequential 3.9 IMAGE SSP2 Base 2045
                   ]

sheet_names_geoloc = ['ei39_CO2','ei39_CH4','ei39_N2O', # Baseline (Swedish grid mix / RER waste treatment)
                      'ei39_SE_CO2', 'ei39_SE_CH4', 'ei39_SE_N2O', # Sweden
                      'ei39_NO_CO2', 'ei39_NO_CH4', 'ei39_NO_N2O', # Norway
                      'ei39_FI_CO2', 'ei39_FI_CH4', 'ei39_FI_N2O', # Finland
                      'ei39_DK_CO2', 'ei39_DK_CH4', 'ei39_DK_N2O', # Denmark
                      'ei39_FR_CO2', 'ei39_FR_CH4', 'ei39_FR_N2O', # France
                      'ei39_DE_CO2', 'ei39_DE_CH4', 'ei39_DE_N2O', # Germany
                      'ei39_BE_CO2', 'ei39_BE_CH4', 'ei39_BE_N2O', # Belgium
                      'ei39_LU_CO2', 'ei39_LU_CH4', 'ei39_LU_N2O', # Luxembourg
                      'ei39_NL_CO2', 'ei39_NL_CH4', 'ei39_NL_N2O', # Netherlands
                      'ei39_AT_CO2', 'ei39_AT_CH4', 'ei39_AT_N2O', # Austria
                      'ei39_RER_CO2', 'ei39_RER_CH4', 'ei39_RER_N2O', # Europe
                     ]

sheet_names_losses = ['ei39_0%_CO2','ei39_0%_CH4','ei39_0%_N2O', # Baseline, 0% change in losses
                      'ei39_20%_CO2','ei39_20%_CH4','ei39_20%_N2O', # +20% losses in the biorefinery processes
                      'ei39_40%_CO2','ei39_40%_CH4','ei39_40%_N2O', # +40% losses in the biorefinery processes
                      'ei39_60%_CO2','ei39_60%_CH4','ei39_60%_N2O', # +60% losses in the biorefinery processes
                      'ei39_80%_CO2','ei39_80%_CH4','ei39_80%_N2O' # +80% losses in the biorefinery processes    
]

In [ ]:
inventory_dict = {}

for name in sheet_names_bg: 
    df = pd.read_excel(str(str(wdir) + '/indirect_GHG_emissions_bg.xlsx'), sheet_name=name, index_col=0)
    inventory_dict[name] = df


for name in sheet_names_geoloc: 
    df = pd.read_excel(str(str(wdir) + '/indirect_GHG_emissions_geoloc.xlsx'), sheet_name=name, index_col=0)
    inventory_dict[name] = df

for name in sheet_names_losses: 
    df = pd.read_excel(str(str(wdir) + '/indirect_GHG_emissions_losses.xlsx'), sheet_name=name, index_col=0)
    inventory_dict[name] = df

inventory_dict.keys()

In [ ]:
inventory_dict['ei39_FR_CH4']

## 2.2 Calculate the time-dependent climate impacts for indirect CO2, CH4, and N2O emissions

The time-dependent climate impacts are calculated for CO2, CH4, and N2O separately based on the inventory data imported (see above). The indirect emissions are multiplied by the DCFs corresponding to the specific year and GHG. The values obtained correspond to the spreadsheets "DCFtotal_ei39" and "DCFtotal_ei39R2".

In [ ]:
# copy the dictionary to store the temporary time-dependent / dynamic climate impacts
DCI_dict = {} 

for df_name in inventory_dict.keys(): 

    # important to create a copy, else updates the content of inventory_dict as well
    df = inventory_dict[df_name].copy()

    if 'CO2' in df_name: 
        df['CF_CO2'] = df.apply(lambda row: fCO2(row['Year']), axis=1)
        df['Stand-alone biorefinery'] = df['Stand-alone biorefinery']*df['CF_CO2']
        df['Integrated biorefinery'] = df['Integrated biorefinery']*df['CF_CO2']
        df['Combustion'] = df['Combustion']*df['CF_CO2']
        #df = df.drop(['Year', 'CF_CO2'], axis=1)
        df = df.drop(['CF_CO2'], axis=1)
        #df.loc['Total']= df.sum()

    if 'CH4' in df_name: 
        df['CF_CH4'] = df.apply(lambda row: fCH4(row['Year']), axis=1)
        df['Stand-alone biorefinery'] = df['Stand-alone biorefinery']*df['CF_CH4']
        df['Integrated biorefinery'] = df['Integrated biorefinery']*df['CF_CH4']
        df['Combustion'] = df['Combustion']*df['CF_CH4']
        #df = df.drop(['Year', 'CF_CH4'], axis=1)
        df = df.drop(['CF_CH4'], axis=1)
        #df.loc['Total']= df.sum()

    if 'N2O' in df_name:
        df['CF_N2O'] = df.apply(lambda row: fN2O(row['Year']), axis=1)
        df['Stand-alone biorefinery'] = df['Stand-alone biorefinery']*df['CF_N2O']
        df['Integrated biorefinery'] = df['Integrated biorefinery']*df['CF_N2O']
        df['Combustion'] = df['Combustion']*df['CF_N2O']
        #df = df.drop(['Year', 'CF_N2O'], axis=1)
        df = df.drop(['CF_N2O'], axis=1)
        #df.loc['Total']= df.sum()

    DCI_dict[df_name] = df

DCI_dict.keys()

In [ ]:
DCI_dict['R2Pk_CH4']

## 4.3. Calculate the total time-dependent climate impacts for indirect CO2, CH4, and N2O emissions

The values obtained for each GHG assessed (CO2, CH4, and N2O) are summed. 

In [ ]:
sum_indirect_DCI = {}
scenario_GHGs = {"ei39": ["ei39_CO2", "ei39_CH4", "ei39_N2O"],"R2Pk": ["R2Pk_CO2", "R2Pk_CH4", "R2Pk_N2O"],"R2B": ["R2B_CO2", "R2B_CH4", "R2B_N2O"],"R1Pk": ["R1Pk_CO2", "R1Pk_CH4", "R1Pk_N2O"],"I2B": ["I2B_CO2", "I2B_CH4", "I2B_N2O"],
                 "ei39SE": ["ei39_SE_CO2", "ei39_SE_CH4", "ei39_SE_N2O"],
                 "ei39NO": ["ei39_NO_CO2", "ei39_NO_CH4", "ei39_NO_N2O"],
                 "ei39FI": ["ei39_FI_CO2", "ei39_FI_CH4", "ei39_FI_N2O"],
                 "ei39DK": ["ei39_DK_CO2", "ei39_DK_CH4", "ei39_DK_N2O"],
                 "ei39FR": ["ei39_FR_CO2", "ei39_FR_CH4", "ei39_FR_N2O"],
                 "ei39DE": ["ei39_DE_CO2", "ei39_DE_CH4", "ei39_DE_N2O"],
                 "ei39BE": ["ei39_BE_CO2", "ei39_BE_CH4", "ei39_BE_N2O"],
                 "ei39LU": ["ei39_LU_CO2", "ei39_LU_CH4", "ei39_LU_N2O"],
                 "ei39NL": ["ei39_NL_CO2", "ei39_NL_CH4", "ei39_NL_N2O"],
                 "ei39AT": ["ei39_AT_CO2", "ei39_AT_CH4", "ei39_AT_N2O"],
                 "ei39RER": ["ei39_RER_CO2", "ei39_RER_CH4", "ei39_RER_N2O"],
                 "ei390%": ['ei39_0%_CO2','ei39_0%_CH4','ei39_0%_N2O'],
                 "ei3920%": ['ei39_20%_CO2','ei39_20%_CH4','ei39_20%_N2O'],
                 "ei3940%": ['ei39_40%_CO2','ei39_40%_CH4','ei39_40%_N2O'],
                 "ei3960%": ['ei39_60%_CO2','ei39_60%_CH4','ei39_60%_N2O'],
                 "ei3980%": ['ei39_80%_CO2','ei39_80%_CH4','ei39_80%_N2O'],
                }

year_col = DCI_dict['ei39_CO2']['Year'] # years are the same for all BGs
for scenario, GHGs in scenario_GHGs.items():
    sum_indirect_DCI[scenario] = sum(DCI_dict[GHG] for GHG in GHGs)
    sum_indirect_DCI[scenario]["Year"] = year_col
sum_indirect_DCI.keys()

In [ ]:
sum_indirect_DCI['ei39SE']

# 3. Time-dependent climate impacts from direct CO2 emissions 

The calculation of the direct CO2-eq emissions is based on the carbon balance of the biorefinery (see SM1 and SM3). Carbon capture is excluded here since the LCA study is limited to gate-to-gate system boundaries (see main manuscript). 

In [ ]:
carbon_content_df = pd.read_excel(str(str(wdir) + '/carbon_content.xlsx'), index_col=0)
carbon_content_df

In [ ]:
direct_CO2_emissions_df = carbon_content_df.copy()
direct_CO2_emissions_df['Stand-alone biorefinery'] = direct_CO2_emissions_df['Stand-alone biorefinery']/1000*3.67
direct_CO2_emissions_df['Integrated biorefinery'] = direct_CO2_emissions_df['Integrated biorefinery']/1000*3.67
direct_CO2_emissions_df['Combustion'] = direct_CO2_emissions_df['Combustion']/1000*3.67
direct_CO2_emissions_df

In [ ]:
sum_direct_DCI = direct_CO2_emissions_df.copy()
# add a column with the dynamic characterisation factors corresponding to the EoL year
sum_direct_DCI['CF_CO2'] = sum_direct_DCI.apply(lambda row: fCO2(row['Year']), axis=1)
sum_direct_DCI['Stand-alone biorefinery'] = sum_direct_DCI['Stand-alone biorefinery']*sum_direct_DCI['CF_CO2']
sum_direct_DCI['Integrated biorefinery'] = sum_direct_DCI['Integrated biorefinery']*sum_direct_DCI['CF_CO2']
sum_direct_DCI['Combustion'] = sum_direct_DCI['Combustion']*sum_direct_DCI['CF_CO2']
#sum_direct_DCI = sum_direct_DCI.drop(['Year', 'CF_CO2'], axis=1)
sum_direct_DCI = sum_direct_DCI.drop(['CF_CO2'], axis=1)
#sum_direct_DCI.loc['Total']= sum_direct_DCI.sum()
sum_direct_DCI

# 4. Time-dependent impacts over 100 years (cf. Figure 4)

Add the values together and make a file with values from 0 to 100 with zeros if no emissions on the specific year. 

## 4.1. Considering direct emissions only (CO2)

In [ ]:
sum_direct_DCI_copy = sum_direct_DCI.copy()
new_row = pd.DataFrame({'Year': [0],'Stand-alone biorefinery': [-1.83],'Integrated biorefinery': [-1.83],'Combustion': [-1.83]}, index=['Carbon capture']) 
sum_direct_DCI_copy = pd.concat([new_row, sum_direct_DCI_copy])
sum_direct_DCI_copy.sort_values(by='Year', ascending=True, inplace=True)
sum_direct_DCI_copy

In [ ]:
# sum the values per year
df_summed = sum_direct_DCI_copy.groupby('Year').sum()

# create df with all years from 0 to 100
all_years = pd.DataFrame({'Year': range(0, 101)})

# merge with the original df to add rows for missing years with zeros
df_full = pd.merge(all_years, df_summed, on='Year', how='left')

# fill missing values with zeros
df_full[['Stand-alone biorefinery', 'Integrated biorefinery', 'Combustion']] = df_full[['Stand-alone biorefinery', 'Integrated biorefinery', 'Combustion']].fillna(0)

# sort the dataframe by year
df_full.sort_values(by='Year', ascending=True, inplace=True)

# export the df to Excel
#df_full.to_excel(str(result_dir+'/total_impacts_over100years_directCO2.xlsx')) 
df_full

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec

# Define color codes for the bars (one color for each scenario)
colors = ['#332288', '#44AA99', '#CC6677']

# Create the main figure and a GridSpec layout with three panels
fig = plt.figure(figsize=(12, 5))
spec = gridspec.GridSpec(1, 3, figure=fig, wspace=0.1, width_ratios=[0.1, 1, 0.1])

# Create subplots for the three panels
ax1 = fig.add_subplot(spec[0, 0])  # First panel
ax2 = fig.add_subplot(spec[0, 1])  # Second panel
ax3 = fig.add_subplot(spec[0, 2])  # Third panel

# Bar settings
bar_width = 0.25  # Width of each bar
x_positions = np.arange(len(df_full["Year"]))  # X-axis positions for the bars (based on the time column)

# Plot bars for each scenario in each panel
for ax in fig.axes:  # Loop over each subplot (ax1, ax2, ax3)
    for i, scenario in enumerate(df_full.columns[1:]):  # Loop over all scenarios (columns except 't')
        y_vals = df_full[scenario]  # Extract data for the current scenario
        valid_idx = ~df_full[scenario].isna()  # Create a mask for valid (non-NaN) values
        ax.bar(
            x_positions[valid_idx] + i * bar_width,  # Offset each scenario by i * bar_width
            y_vals[valid_idx],                      # Y-values for the bars
            width=bar_width,                        # Bar width
            label=scenario,                         # Scenario name for the legend
            color=colors[i],                        # Bar color
            edgecolor='black',                      # Outline color for the bars
            alpha=0.8                               # Bar transparency
        )

# Customize x-axis limits for each panel to focus on specific ranges of data
ax1.set_xlim(-0.5, 1.5)  # Focus on the first time segment
ax2.set_xlim(33, 49)     # Focus on the middle time segment
ax3.set_xlim(84.5, 86)   # Focus on the last time segment

# Add axis labels
ax2.set_xlabel("Time (year)", fontsize=11)  # X-axis label
ax1.set_ylabel("Time-dependent climate impacts (kgCO2-eq/FU/year)", fontsize=11)  # Y-axis label for the first panel

# Add legend to the second panel
ax2.legend(frameon=False, bbox_to_anchor=(1, 1),loc='upper right', fontsize=12)

# Remove unnecessary spines and y-ticks for cleaner appearance
ax1.spines.right.set_visible(False)  # Hide the right spine for the first panel
ax2.spines.left.set_visible(False)   # Hide the left spine for the second panel
ax2.spines.right.set_visible(False)  # Hide the right spine for the second panel
ax3.spines.left.set_visible(False)   # Hide the left spine for the third panel

# Remove unnecessary spines for cleaner appearance
for ax in [ax1, ax2, ax3]:
    ax.spines['right'].set_visible(False)  # Hide the right spine
    ax.spines['top'].set_visible(False)    # Hide the top spine

ax2.set_yticks([])  # Remove y-ticks for the second panel
ax3.set_yticks([])  # Remove y-ticks for the third panel

# Add diagonal lines to indicate breaks in the x-axis
d = 0.02  # Size of the diagonal lines in axis coordinates

# Panel 1: Bottom-right diagonal
kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
ax1.plot((1, 1), (d, -d), **kwargs, zorder=101, lw=0.5)

# Panel 2: Bottom-left and bottom-right diagonals
kwargs = dict(transform=ax2.transAxes, color='k', clip_on=False)
ax2.plot((0, 0), (d, -d), **kwargs, zorder=101, lw=0.5)  # Bottom-left diagonal
ax2.plot((1, 1), (d, -d), **kwargs, zorder=101, lw=0.5)  # Bottom-right diagonal

# Set tick parameters for font size
for ax in fig.axes:
    ax.tick_params(axis='x', labelsize=13)  # Increase x-axis tick font size
    ax.tick_params(axis='y', labelsize=13)  # Increase y-axis tick font size

# Panel 3: Bottom-left diagonal
kwargs = dict(transform=ax3.transAxes, color='k', clip_on=False)
ax3.plot((0, 0), (d, -d), **kwargs, zorder=101, lw=0.5)

# Adjust layout to ensure proper spacing
plt.tight_layout()
plt.savefig(str(result_dir+'/fig4_climate_impacts_DCFs_direct_emissions.pdf'), dpi=300, bbox_inches='tight')
# Display the plot
plt.show()

## 4.2. Considering direct (CO2) and indirect emissions (CO2, CH4, N2O)
The indirect emissions were calculated using the REMIND SSP2 PkBudg500 2045 background database. This calculation can be done with any of the background databases used in the Part 1. 

In [ ]:
sum_direct_DCI_copy = sum_direct_DCI.copy()
sum_indirect_DCI_R2Pk_copy = sum_indirect_DCI['R2Pk'].copy()
sum_indirect_DCI_R2Pk_copy.rename_axis('Product', inplace=True)

In [ ]:
# merge the two dataframes
df_combined = pd.merge(sum_direct_DCI_copy, sum_indirect_DCI_R2Pk_copy, on=['Product', 'Year'], how='outer', suffixes=('_dir', '_ind'))

# add the values of the matching columns
df_combined['Stand-alone biorefinery'] = df_combined['Stand-alone biorefinery_dir'].fillna(0) + df_combined['Stand-alone biorefinery_ind'].fillna(0)
df_combined['Integrated biorefinery'] = df_combined['Integrated biorefinery_dir'].fillna(0) + df_combined['Integrated biorefinery_ind'].fillna(0)
df_combined['Combustion'] = df_combined['Combustion_dir'].fillna(0) + df_combined['Combustion_ind'].fillna(0)

# drop the separate columns
df_combined.drop(columns=['Stand-alone biorefinery_dir', 'Stand-alone biorefinery_ind', 
                          'Integrated biorefinery_dir', 'Integrated biorefinery_ind', 
                          'Combustion_dir', 'Combustion_ind'], inplace=True)

# add carbon capture during Year 0
new_row = pd.DataFrame({'Year': [0],'Stand-alone biorefinery': [-1.83],'Integrated biorefinery': [-1.83],'Combustion': [-1.83]}, index=['Carbon capture']) 

# Append the new row to the original DataFrame
df_combined = pd.concat([new_row, df_combined])

df_combined.sort_values(by='Year', ascending=True, inplace=True)
df_combined

In [ ]:
# sum the values per year
df_summed_total = df_combined.groupby('Year').sum()

# create df with all years from 0 to 100
all_years = pd.DataFrame({'Year': range(0, 101)})

# merge with the original df to add rows for missing years with zeros
df_full_total = pd.merge(all_years, df_summed_total, on='Year', how='left')

# fill missing values with zeros
df_full_total[['Stand-alone biorefinery', 'Integrated biorefinery', 'Combustion']] = df_full_total[['Stand-alone biorefinery', 'Integrated biorefinery', 'Combustion']].fillna(0)

# sort the dataframe by year
df_full_total.sort_values(by='Year', ascending=True, inplace=True)

# export the df to Excel
#df_full_total.to_excel(str(result_dir+'/total_impacts_over100years_allGHGs.xlsx')) 
df_full_total

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec

# Define color codes for the bars (one color for each scenario)
colors = ['#332288', '#44AA99', '#CC6677']

# Create the main figure and a GridSpec layout with three panels
fig = plt.figure(figsize=(12, 5))
spec = gridspec.GridSpec(1, 3, figure=fig, wspace=0.1, width_ratios=[0.1, 1, 0.1])

# Create subplots for the three panels
ax1 = fig.add_subplot(spec[0, 0])  # First panel
ax2 = fig.add_subplot(spec[0, 1])  # Second panel
ax3 = fig.add_subplot(spec[0, 2])  # Third panel

# Bar settings
bar_width = 0.25  # Width of each bar
x_positions = np.arange(len(df_full_total["Year"]))  # X-axis positions for the bars (based on the time column)

# Plot bars for each scenario in each panel
for ax in fig.axes:  # Loop over each subplot (ax1, ax2, ax3)
    for i, scenario in enumerate(df_full_total.columns[1:]):  # Loop over all scenarios (columns except 't')
        y_vals = df_full_total[scenario]  # Extract data for the current scenario
        valid_idx = ~df_full_total[scenario].isna()  # Create a mask for valid (non-NaN) values
        ax.bar(
            x_positions[valid_idx] + i * bar_width,  # Offset each scenario by i * bar_width
            y_vals[valid_idx],                      # Y-values for the bars
            width=bar_width,                        # Bar width
            label=scenario,                         # Scenario name for the legend
            color=colors[i],                        # Bar color
            edgecolor='black',                      # Outline color for the bars
            alpha=0.8                               # Bar transparency
        )

# Customize x-axis limits for each panel to focus on specific ranges of data
ax1.set_xlim(-0.5, 1.5)  # Focus on the first time segment
ax2.set_xlim(33, 49)     # Focus on the middle time segment
ax3.set_xlim(84.5, 86)   # Focus on the last time segment

# Add axis labels
ax2.set_xlabel("Time (year)", fontsize=11)  # X-axis label
ax1.set_ylabel("Time-dependent climate impacts (kgCO2-eq/FU/year)", fontsize=11)  # Y-axis label for the first panel

# Add legend to the second panel
#ax2.legend(frameon=False, bbox_to_anchor=(1, 0.07),loc='lower right', fontsize=13)

# Remove unnecessary spines and y-ticks for cleaner appearance
ax1.spines.right.set_visible(False)  # Hide the right spine for the first panel
ax2.spines.left.set_visible(False)   # Hide the left spine for the second panel
ax2.spines.right.set_visible(False)  # Hide the right spine for the second panel
ax3.spines.left.set_visible(False)   # Hide the left spine for the third panel

# Remove unnecessary spines for cleaner appearance
for ax in [ax1, ax2, ax3]:
    ax.spines['right'].set_visible(False)  # Hide the right spine
    ax.spines['top'].set_visible(False)    # Hide the top spine

ax2.set_yticks([])  # Remove y-ticks for the second panel
ax3.set_yticks([])  # Remove y-ticks for the third panel

# Add diagonal lines to indicate breaks in the x-axis
d = 0.02  # Size of the diagonal lines in axis coordinates

# Panel 1: Bottom-right diagonal
kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
ax1.plot((1, 1), (d, -d), **kwargs, zorder=101, lw=0.5)

# Panel 2: Bottom-left and bottom-right diagonals
kwargs = dict(transform=ax2.transAxes, color='k', clip_on=False)
ax2.plot((0, 0), (d, -d), **kwargs, zorder=101, lw=0.5)  # Bottom-left diagonal
ax2.plot((1, 1), (d, -d), **kwargs, zorder=101, lw=0.5)  # Bottom-right diagonal

# Set tick parameters for font size
for ax in fig.axes:
    ax.tick_params(axis='x', labelsize=13)  # Increase x-axis tick font size
    ax.tick_params(axis='y', labelsize=13)  # Increase y-axis tick font size

# Panel 3: Bottom-left diagonal
kwargs = dict(transform=ax3.transAxes, color='k', clip_on=False)
ax3.plot((0, 0), (d, -d), **kwargs, zorder=101, lw=0.5)

# Adjust layout to ensure proper spacing
plt.tight_layout()
plt.savefig(str(result_dir+'/fig4_climate_impacts_DCFs_total_emissions.pdf'), dpi=300, bbox_inches='tight')
# Display the plot
plt.show()


# 5. Total time-dependent climate impacts

The code below was used to generate Figure 5 from the main article. 

In [ ]:
sum_indirect_DCI.keys()

In [ ]:
# store all BGs in a dict
bg_df_dict_temp = {'ei39':sum_indirect_DCI['ei39'], 'R2Pk': sum_indirect_DCI['R2Pk'], 'R2B': sum_indirect_DCI['R2B'], 'R1Pk': sum_indirect_DCI['R1Pk'], 'I2B': sum_indirect_DCI['I2B'],
                   'ei39SE':sum_indirect_DCI['ei39SE'],
                   'ei39NO':sum_indirect_DCI['ei39NO'],
                   'ei39FI':sum_indirect_DCI['ei39FI'],
                   'ei39DK':sum_indirect_DCI['ei39DK'],
                   'ei39FR':sum_indirect_DCI['ei39FR'],
                   'ei39DE':sum_indirect_DCI['ei39DE'],
                   'ei39BE':sum_indirect_DCI['ei39BE'],
                   'ei39LU':sum_indirect_DCI['ei39LU'],
                   'ei39NL':sum_indirect_DCI['ei39NL'],
                   'ei39AT':sum_indirect_DCI['ei39AT'],
                   'ei39RER':sum_indirect_DCI['ei39RER'], 
                   'ei390%':sum_indirect_DCI['ei390%'], 
                   'ei3920%':sum_indirect_DCI['ei3920%'], 
                   'ei3940%':sum_indirect_DCI['ei3940%'], 
                   'ei3960%':sum_indirect_DCI['ei3960%'], 
                   'ei3980%':sum_indirect_DCI['ei3980%'], 
                  }

# drop the 'Year' column for all the dfs
bg_df_dict = {}
for db_name in bg_df_dict_temp.keys():
    df = bg_df_dict_temp[db_name].copy()
    df = df.drop(['Year'], axis=1) # drop the 'Year' column
    bg_df_dict[db_name] = df
bg_df_dict.keys()

# add a 'Total' row to the direct impacts df
sum_direct_DCI.loc['Total']= sum_direct_DCI.sum()

In [ ]:
sum_direct_DCI

In [ ]:
# choose the bg to plot
ei = 'ei39'
#ei = 'R2Pk'
total_df = bg_df_dict[ei] 

In [ ]:
# aggregate the EoL values into a unique row
total_df.loc['EoL']= total_df.iloc[10:20].sum()
total_df_vf = total_df.drop(total_df.index[10:20])

# move the EoL row up
#total_df_vf = total_df_vf.drop(['Total'])
row_to_move = total_df_vf.iloc[[10]]
total_df_vf = total_df_vf.drop(['EoL'])
total_df_vf = pd.concat([total_df_vf.iloc[:5], row_to_move, total_df_vf.iloc[5:]])

## add a row from the direct CO2 emissions (biogenic)
total_df_vf.loc['direct CO2'] = sum_direct_DCI.loc['Total']

#change index labels
index_labels = ['lipophilic extraction', 'sugar extraction', 'pulping', 'heat (RB)', 'heat (burd.)','EoL', 
                'tall oil (sub.)', 'starch (sub.)', 'phenol (sub.)', 'pulp (sub.)', 'heat CHP bark', 'biogenic CO2']
total_df_vf.index = index_labels
total_df_vf.rename(columns={'Stand-alone biorefinery': 'Stand-alone\nbiorefinery', 'Integrated biorefinery': 'Integrated\nbiorefinery'}, inplace=True)
#total_df_vf.loc['Total']= total_df_vf.sum()
total_df_vf.to_excel(str(result_dir+'/total_climate_impacts_DCFs_' + ei +'.xlsx')) 
total_df_vf

In [ ]:
# color palette from: https://davidmathlogic.com/colorblind/#%23332288-%23117733-%2344AA99-%2388CCEE-%23DDCC77-%23CC6677-%23AA4499-%23882255
colors = ['#DDCC77', # LE yellow
          '#88CCEE', # SE sky blue
          '#332288', # P dark blue
          '#fff0db', # heat (RB) beige
          'black', # heat burd. black
          '#882255',# EoL red wine
          '#D3D3D3', #tall oil sub.light grey
          '#44AA99', # starch sub. teal
          '#117733', # phenol sub. dark green 
          '#AA4499', # pulp sub.plum pink
          'grey', # heat CHP bark dark grey
          '#CC6677', # biogenic CO2 orange pink          
]

In [ ]:
# GWP scores with substitution
GWP_scores_ei39 = list(total_df_vf.sum())
GWP_scores_ei39 = ['%.2f' % elem for elem in GWP_scores_ei39]
GWP_scores_ei39 = [float(elem) for elem in GWP_scores_ei39] # the list items must be floats
GWP_scores_ei39

# GWP scores without substitution
df_wo_subst_ei39 = total_df_vf.drop(['heat (burd.)', 'tall oil (sub.)', 'starch (sub.)', 'phenol (sub.)', 'pulp (sub.)'])
GWP_scores_wo_subst_ei39 = list(df_wo_subst_ei39.sum())
GWP_scores_wo_subst_ei39 = ['%.2f' % elem for elem in GWP_scores_wo_subst_ei39]
GWP_scores_wo_subst_ei39 = [float(elem) for elem in GWP_scores_wo_subst_ei39] # the list items must be floats
GWP_scores_wo_subst_ei39

# transpose the dataframe
df_transposed = total_df_vf.T

fig, ax = plt.subplots(figsize=(7, 7))

ax.set_ylim(-0.7,3.0)

col_names = ['Stand-alone\nbiorefinery', 'Integrated\nbiorefinery', 'Combustion']

bars = df_transposed.plot(kind='bar', stacked=True, ax=ax, color=colors,legend = False, width=0.5)
ax.set_ylabel('Time-dependent climate impacts (kg CO2-eq/FU)', fontsize=14)
#ax.legend(loc='lower center', bbox_to_anchor=(0.55, 0.7), title_fontsize='12', fontsize='14', ncol=2, frameon = False)

ax.tick_params(axis='x', labelsize=14, rotation=0)
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', which='both', bottom=False)  # Remove x-axis ticks

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# add the single GWP100 scores for each scenario
categories_index = np.arange(len(col_names))
ax.scatter(categories_index, GWP_scores_wo_subst_ei39, color='black', marker = '^', zorder=5)
ax.scatter(categories_index, GWP_scores_ei39, color='black', marker = 'H', zorder=5)

for i, val in enumerate(GWP_scores_ei39):
    ax.text(categories_index[i], val - 0.12, f'{val}', ha='center', color='black',va = 'top', fontsize=14)

for i, val in enumerate(GWP_scores_wo_subst_ei39):
    ax.text(categories_index[i], val+0.12, f'{val}', ha='center', color='black',va = 'bottom', fontsize=14)

# +/- 0.08

plt.tight_layout()

#plt.savefig(str(result_dir+'/fig5_DCFs_results_' + ei + '.pdf'), dpi=300)
plt.savefig(str(result_dir+'/fig5_DCFs_results_'+ei+'.pdf'), dpi=300)

plt.show()

# 6. Sensitivity analysis

## 6.1. Influence of the product's lifespan

The product's lifespan influences the calculation of the time-dependent climate impacts through the choice of dynamic characterisation factors. Here, the change in losses of biomass are fixed to 0 (i.e. one-at-a-time sensitivity analysis). Both the direct and indirect emissions are affected by a change in the dynamic CFs. The maximum lifespan is fixed at 100 years. The background database used is ecoinvent consequential 3.9

In [ ]:
percentage_change_list = np.arange(-0.3, 0.3,0.01).tolist()
#percentage_change_list = [0] # to check that the values obtained are the same than in the Excel file
inventory_dict.keys()

In [ ]:
def calculate_time_dependent_impacts_GHG(emissions_df, GHG, percentage_change_list, f):

    GHG_impact_standalone = []
    GHG_impact_integrated = []
    GHG_impact_combustion = []

    for value in percentage_change_list:
        
        df = emissions_df.copy()

        # Adjust the "Year" values based on the percentage change
        df['Year'] = df['Year'] + df['Year'] * value
        df['Year'] = df['Year'].apply(lambda x: min(x, 100))  # Cap at 100 years

        # Calculate the climate factor and impacts
        cf_column = f'CF_{GHG}'
        df[cf_column] = df.apply(lambda row: f(row['Year']), axis=1)
        df['Stand-alone biorefinery'] *= df[cf_column]
        df['Integrated biorefinery'] *= df[cf_column]
        df['Combustion'] *= df[cf_column]

        # Calculate total impacts
        GHG_impact_standalone.append(df['Stand-alone biorefinery'].sum())
        GHG_impact_integrated.append(df['Integrated biorefinery'].sum())
        GHG_impact_combustion.append(df['Combustion'].sum())

    return GHG_impact_standalone, GHG_impact_integrated, GHG_impact_combustion

In [ ]:
def calculate_total_impacts():

    total_impact_standalone = []
    total_impact_integrated = []
    total_impact_combustion = []

    iSCO2, iICO2, iCCO2 = calculate_time_dependent_impacts_GHG(inventory_dict['ei39_CO2'], 'CO2', percentage_change_list, fCO2)
    SCH4, ICH4, CCH4 = calculate_time_dependent_impacts_GHG(inventory_dict['ei39_CH4'], 'CH4', percentage_change_list, fCH4)
    SN2O, IN2O, CN2O = calculate_time_dependent_impacts_GHG(inventory_dict['ei39_N2O'], 'N2O', percentage_change_list, fN2O)
    dSCO2, dICO2, dCCO2 = calculate_time_dependent_impacts_GHG(direct_CO2_emissions_df, 'CO2', percentage_change_list, fCO2)
    
    # Sum indirect and direct impacts
    total_standalone = [a + b + c + d for a,b,c,d in zip(iSCO2,SCH4,SN2O,dSCO2)]
    total_integrated = [a + b + c + d for a,b,c,d in zip(iICO2,ICH4,IN2O,dICO2)]
    total_combustion = [a + b + c + d for a,b,c,d in zip(iCCO2,CCH4,CN2O, dCCO2)]

    return total_standalone, total_integrated, total_combustion

In [ ]:
total_standalone, total_integrated, total_combustion = calculate_total_impacts()

In [ ]:
import matplotlib.pyplot as plt

# Plotting the data
fig, ax = plt.subplots(figsize=(7, 5))

# Plot continuous lines
plt.plot(percentage_change_list, total_standalone, 
         label='Stand-alone biorefinery', color='#332288', linestyle='-', linewidth=3.5)
plt.plot(percentage_change_list, total_integrated, 
         label='Integrated biorefinery', color='#44AA99', linestyle='-', linewidth=3.5)
plt.plot(percentage_change_list, total_combustion, 
         label='Combustion', color='#CC6677', linestyle='-', linewidth=3.5)

# Adding titles and labels
plt.xlabel('Change of lifespan (%)', size=13)
plt.ylabel('Time-dependent climate impacts (kg CO2-eq/FU)', size=13)
plt.legend(frameon=False, fontsize=13, loc='lower center', bbox_to_anchor=(0.27, 0.5))

# Customize axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', labelsize=13, rotation=0)
ax.tick_params(axis='y', labelsize=13)

# Save and show plot
plt.tight_layout()
plt.savefig(str(result_dir+'/fig6_sensitivity_lifespan_DCFs'+ei+'.pdf'), dpi=300)

plt.show()

## 6.2. Influence of the background database

The choice of the background database influences the calculation of the indirect emissions. The results obtained are stored in Excel files that are imported below to plot the sensitivity analysis graph. 

In [ ]:
bg_df_dict['I2B']

In [ ]:
bg_df_dict_agg = {}
df_total_impacts = pd.DataFrame(columns=['Stand-alone\nbiorefinery','Integrated\nbiorefinery','Combustion'])
list_bg = ['ei39', 'R2Pk', 'R2B', 'R1Pk', 'I2B']

for key in list_bg:

    df = bg_df_dict[key].copy()

    # aggregate the EoL values into a unique row
    df.loc['EoL']= df.iloc[10:20].sum()
    df = df.drop(df.index[10:20])

    # move the EoL row up
    row_to_move = df.iloc[[10]]
    df = df.drop(['EoL'])
    df = pd.concat([df.iloc[:5], row_to_move, df.iloc[5:]])
    
    ## add a row from the direct CO2 emissions (biogenic)
    df.loc['direct CO2'] = sum_direct_DCI.loc['Total']

    df.loc['Total'] = df.sum()
    
    #change index labels
    index_labels = ['lipophilic extraction', 'sugar extraction', 'pulping', 'heat (RB)', 'heat (burd.)','EoL', 
                    'tall oil (sub.)', 'starch (sub.)', 'phenol (sub.)', 'pulp (sub.)', 'heat CHP bark', 'biogenic CO2', 'Total']
    
    df.index = index_labels
    df.rename(columns={'Stand-alone biorefinery': 'Stand-alone\nbiorefinery', 'Integrated biorefinery': 'Integrated\nbiorefinery'}, inplace=True)
    #total_df_vf.loc['Total']= total_df_vf.sum()

    # export the results to Excel file
    df.to_excel(str(result_dir+'/total_climate_impacts_DCFs_' + key +'.xlsx')) 

    # add df to updated dict
    bg_df_dict_agg[key]=df 

    # add total values to df
    df_total_impacts.loc[key] = df.loc['Total'] 

bg_df_dict_agg.keys()

In [ ]:
df_total_impacts

In [ ]:
# Marker styles and colors for different models
markers = ['o', 's', '^', 'D', 'p']
colors = ['#332288', '#44AA99', '#88CCEE', '#DDCC77', '#CC6677']
scenarios= ['Stand-alone\nbiorefinery','Integrated\nbiorefinery','Combustion']
db_names = {'ei39':'Conventional', 'R2Pk':'REMIND-SSP2-PkBudg500-2045', 'R2B':'REMIND-SSP2-Base-2045','R1Pk':'REMIND-SSP1-PkBudg500-2045','I2B':'IMAGE-SSP2-Base-2045'}

# Create figure and axis
fig, ax = plt.subplots(figsize=(7,5))

# Set the positions of the x-ticks with reduced spacing
x_positions = (np.arange(len(scenarios))*4)  # Reduced spacing

# Plotting data points for each model in all scenarios
for j, db in enumerate(db_names.keys()):
    x_coords = []
    y_coords = []
    for i, scenario in enumerate(scenarios):
        x_coords.append(x_positions[i])  # Adjusted x-axis position
        y_coords.append(df_total_impacts[scenario][db])  # Corresponding y-value for the model

    # Plot each model with a unique marker and color
    plt.scatter(x_coords, y_coords, marker=markers[j], facecolor = 'None',edgecolor=colors[j], s=100, label=db_names[db], linewidths=3)
    #plt.scatter(x_coords, y_coords, marker=markers[j], color=colors[j], s=100, label=model)

# Set labels and title#
plt.xticks(ticks=x_positions, labels=scenarios, size=14)
ax.set_xlim(-1.5,10)
#plt.xlabel("Scenarios")
plt.ylabel("Time-dependent climate impacts (kg CO2-eq/FU)", size = 13)
#plt.title("Scatter Plot of Different Scenarios and Models")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.tick_params(axis='x', labelsize=13, rotation=0)
ax.tick_params(axis='y', labelsize=13)

# Add legend for models, with a smaller font size and adjusted position
plt.legend(frameon=False,bbox_to_anchor=(0.98, 0.98), loc='upper right', fontsize=13)

# Show grid
#plt.grid(True)

# Adjust layout to make room for the legend
plt.tight_layout()
plt.savefig(str(result_dir+'/fig6_sensitivity_background_DCFs'+ei+'.pdf'), dpi=300)

# Show the plot
plt.show()

## 6.3. Influence of the geographical location

In [ ]:
bg_df_dict_agg = {}
df_total_impacts_geoloc = pd.DataFrame(columns=['Stand-alone\nbiorefinery','Integrated\nbiorefinery','Combustion'])
list_bg = ['ei39','ei39SE', 'ei39NO', 'ei39FI', 'ei39DK', 'ei39FR', 'ei39DE', 'ei39BE', 'ei39LU', 'ei39NL', 'ei39AT', 'ei39RER']

for key in list_bg:

    df = bg_df_dict[key].copy()

    # aggregate the EoL values into a unique row
    df.loc['EoL']= df.iloc[10:20].sum()
    df = df.drop(df.index[10:20])

    # move the EoL row up
    row_to_move = df.iloc[[10]]
    df = df.drop(['EoL'])
    df = pd.concat([df.iloc[:5], row_to_move, df.iloc[5:]])
    
    ## add a row from the direct CO2 emissions (biogenic)
    df.loc['direct CO2'] = sum_direct_DCI.loc['Total']

    df.loc['Total'] = df.sum()
    
    #change index labels
    index_labels = ['lipophilic extraction', 'sugar extraction', 'pulping', 'heat (RB)', 'heat (burd.)','EoL', 
                    'tall oil (sub.)', 'starch (sub.)', 'phenol (sub.)', 'pulp (sub.)', 'heat CHP bark', 'biogenic CO2', 'Total']
    
    df.index = index_labels
    df.rename(columns={'Stand-alone biorefinery': 'Stand-alone\nbiorefinery', 'Integrated biorefinery': 'Integrated\nbiorefinery'}, inplace=True)
    #total_df_vf.loc['Total']= total_df_vf.sum()

    # export the results to Excel file
    df.to_excel(str(result_dir+'/total_climate_impacts_DCFs_' + key +'.xlsx')) 

    # add df to updated dict
    bg_df_dict_agg[key]=df 

    # add total values to df
    df_total_impacts_geoloc.loc[key] = df.loc['Total'] 

bg_df_dict_agg.keys()

In [ ]:
df_total_impacts_geoloc

In [ ]:
# Plotting the data
fig, ax = plt.subplots(figsize=(7, 5))

countries = ['Base.','SE','NO', 'FI', 'DK', 'FR', 'DE', 'BE', 'LU', 'NL', 'AT', 'RER']

plt.scatter(countries, df_total_impacts_geoloc['Stand-alone\nbiorefinery'], label='Stand-alone biorefinery', marker='o', color = '#332288', s=100)
plt.scatter(countries, df_total_impacts_geoloc['Integrated\nbiorefinery'], label='Integrated biorefinery', marker='s', color = '#44AA99', s=100)
plt.scatter(countries, df_total_impacts_geoloc['Combustion'], label='Combustion', marker='^', color = '#CC6677', s=100)

# Adding titles and labels
plt.xlabel('Geographical location (country)', size =13)
plt.ylabel('Time-dependent climate impacts (kg CO2-eq/FU)', size=13)
plt.legend(frameon=False, fontsize=14)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.tick_params(axis='x', labelsize=13, rotation=0)
ax.tick_params(axis='y', labelsize=13)

    
plt.tight_layout()
plt.savefig(str(result_dir+'/fig6_sensitivity_geoloc_'+ei+'.pdf'), dpi=300)


# Show plot
plt.show()

## 6.4. Influence of biomass losses

A change in the percentage of biomass losses affects both the direct and indirect emissions for the biorefinery scenarios (not for the combustion scenario).

In [ ]:
bg_df_dict_agg = {}
df_total_impacts_losses = pd.DataFrame(columns=['Stand-alone\nbiorefinery','Integrated\nbiorefinery','Combustion'])
list_bg = ['ei390%','ei3920%','ei3940%','ei3960%','ei3980%']
percentage_change_list = ['0%', '20%', '40%', '60%', '80%']

for key in list_bg:

    df = bg_df_dict[key].copy()

    # aggregate the EoL values into a unique row
    df.loc['EoL']= df.iloc[10:20].sum()
    df = df.drop(df.index[10:20])

    # move the EoL row up
    row_to_move = df.iloc[[10]]
    df = df.drop(['EoL'])
    df = pd.concat([df.iloc[:5], row_to_move, df.iloc[5:]])
    
    ## add a row from the direct CO2 emissions (biogenic)
    df.loc['direct CO2'] = sum_direct_DCI.loc['Total']

    df.loc['Total'] = df.sum()
    
    #change index labels
    index_labels = ['lipophilic extraction', 'sugar extraction', 'pulping', 'heat (RB)', 'heat (burd.)','EoL', 
                    'tall oil (sub.)', 'starch (sub.)', 'phenol (sub.)', 'pulp (sub.)', 'heat CHP bark', 'biogenic CO2', 'Total']
    
    df.index = index_labels
    df.rename(columns={'Stand-alone biorefinery': 'Stand-alone\nbiorefinery', 'Integrated biorefinery': 'Integrated\nbiorefinery'}, inplace=True)
    #total_df_vf.loc['Total']= total_df_vf.sum()

    # export the results to Excel file
    df.to_excel(str(result_dir+'/total_climate_impacts_DCFs_losses' + key +'.xlsx')) 

    # add df to updated dict
    bg_df_dict_agg[key]=df 

    # add total values to df
    df_total_impacts_losses.loc[key] = df.loc['Total'] 

bg_df_dict_agg.keys()

In [ ]:
df_total_impacts_losses

In [ ]:
# Create figure
fig, ax = plt.subplots(figsize=(7, 5))

# Convert percentage strings to numeric values
percentage_change_numeric = np.array([float(x.strip('%')) for x in percentage_change_list])

# Fit lines using numpy's polyfit
fitted_standalone = np.polyfit(percentage_change_numeric, 
                             df_total_impacts_losses['Stand-alone\nbiorefinery'].astype(float), 1)
fitted_integrated = np.polyfit(percentage_change_numeric, 
                             df_total_impacts_losses['Integrated\nbiorefinery'].astype(float), 1)
fitted_combustion = np.polyfit(percentage_change_numeric, 
                             df_total_impacts_losses['Combustion'].astype(float), 1)

# Create linear functions
line_standalone = np.poly1d(fitted_standalone)
line_integrated = np.poly1d(fitted_integrated)
line_combustion = np.poly1d(fitted_combustion)

# Plot fitted lines
plt.plot(percentage_change_numeric, line_standalone(percentage_change_numeric), 
         label='Stand-alone biorefinery', color='#332288', linestyle='-', linewidth=3.5)
plt.plot(percentage_change_numeric, line_integrated(percentage_change_numeric), 
         label='Integrated biorefinery', color='#44AA99', linestyle='-', linewidth=3.5)
plt.plot(percentage_change_numeric, line_combustion(percentage_change_numeric), 
         label='Combustion', color='#CC6677', linestyle='-', linewidth=3.5)

# Add scatter points
# plt.scatter(percentage_change_numeric, df_total_impacts_losses['Stand-alone\nbiorefinery'], 
#            color='#332288')
# plt.scatter(percentage_change_numeric, df_total_impacts_losses['Integrated\nbiorefinery'], 
#            color='#44AA99')
# plt.scatter(percentage_change_numeric, df_total_impacts_losses['Combustion'], 
#            color='#CC6677')

# Format x-axis to show percentages
ax.set_xticks(percentage_change_numeric)
ax.set_xticklabels([f'{int(x)}%' for x in percentage_change_numeric])

# Adding titles and labels
plt.xlabel('Change of percentage of biomass losses (%)', size=13)
plt.ylabel('Time-dependent climate impacts (kg CO2-eq/FU)', size=13)
plt.legend(frameon=False, fontsize=13, loc='lower center', bbox_to_anchor=(0.27, 0.5))

# Customize axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', labelsize=13, rotation=0)
ax.tick_params(axis='y', labelsize=13)

# Save and show plot
plt.tight_layout()
plt.savefig(str(result_dir+'/fig6_sensitivity_losses_DCFs'+ei+'.pdf'), dpi=300)
plt.show()
